In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt
from calculate_score import *
from collections import defaultdict

In [3]:


config_hand1 = {"a": 70, "b": 145, "base_dir": "hand", "marker_name": ["left_wrist", "right_wrist"], "modes": ["uni", "bi"]}
config_foot1 = {"a": 70, "b": 145, "base_dir": "foot", "marker_name": ["left_ankle", "right_ankle"], "modes": ["uni", "bi"]}

config_hand2 = {"a": 60, "b": 180, "base_dir": "hand", "marker_name": ["left_wrist", "right_wrist"], "modes": ["uni", "bi"]}
config_foot2 = {"a": 60, "b": 180, "base_dir": "foot", "marker_name": ["left_ankle", "right_ankle"], "modes": ["uni", "bi"]}

configs = [config_hand1, config_foot1, config_hand2, config_foot2]

for cfg in configs:
    a = cfg["a"]
    b = cfg["b"]
    base_dir = cfg["base_dir"]
    for marker_name in cfg["marker_name"]:
        for mode in cfg["modes"]:
            read_path = f"./aist_vel1s/z-score/tempo_{a}_{b}/{base_dir}/{marker_name}_zero_{mode}_{a}_{b}.csv"
            save_path = f'./aist_vel1s/z-score/tempo_{a}_{b}/score/{base_dir}/{marker_name}_{mode}_{a}_{b}_score.csv'
            calc_score(mode, read_path, save_path)
            


Score saved to ./aist_vel1s/z-score/tempo_70_145/score/hand/left_wrist_uni_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/hand/left_wrist_bi_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/hand/right_wrist_uni_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/hand/right_wrist_bi_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/foot/left_ankle_uni_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/foot/left_ankle_bi_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/foot/right_ankle_uni_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_70_145/score/foot/right_ankle_bi_70_145_score.csv
Score saved to ./aist_vel1s/z-score/tempo_60_180/score/hand/left_wrist_uni_60_180_score.csv
Score saved to ./aist_vel1s/z-score/tempo_60_180/score/hand/left_wrist_bi_60_180_score.csv
Score saved to ./aist_vel1s/z-score/tempo_60_180/score/hand/right_wrist_uni_60_18

In [ ]:
a = 70; b = 145
base_dir = "hand"
marker_name = "left_wrist"      # "left_ankle", "right_ankle", "left_wrist", "right_wrist"
mode = f"bi_{a}_{b}"

root_dir = f"aist_pos1s/z-score/tempo_{a}_{b}"

df_results = pd.read_csv(f"./{root_dir}/{base_dir}/{marker_name}_zero_{mode}.csv")

ref = df_results["music_tempo"].to_numpy()
bpm_avg_x = df_results[f"bpm_avg_x"].to_numpy()
bpm_avg_y = df_results[f"bpm_avg_y"].to_numpy()
bpm_avg_xy = df_results[f"bpm_avg_xy"].to_numpy()


bpm_mode_x = df_results[f"bpm_mode_x"].to_numpy()
bpm_mode_y = df_results[f"bpm_mode_y"].to_numpy()
bpm_mode_xy = df_results[f"bpm_mode_xy"].to_numpy()

bpm_median_x = df_results[f"bpm_median_x"].to_numpy()
bpm_median_y = df_results[f"bpm_median_y"].to_numpy()
bpm_median_xy = df_results[f"bpm_median_xy"].to_numpy()


### oe1 oe2 acc1 acc2 m3

In [ ]:
# Data for each experiment
experiments = {
    "1s": [bpm_avg_x, bpm_avg_y, bpm_mode_xy, 
           bpm_mode_x, bpm_mode_y, bpm_mode_xy,
           bpm_median_x, bpm_median_y, bpm_median_xy],
            }
axes = ["bpm_avg_x", "bpm_avg_y", "bpm_avg_xy",
        "bpm_mode_x", "bpm_mode_y", "bpm_mode_xy",
        "bpm_median_x", "bpm_median_y", "bpm_median_xy"]

# results = {"direction": [], "axis": [], "acc1": [], "acc2": [], "acc3": [],
#            "hits_idx": [], "hits_dbl_idx": [], "hits_hf_idx": []}

results = defaultdict(list)

tolerance = 8
for exp_name, data in experiments.items():
    for axis_name, calculated in zip(axes, data):
        metrics, hits_idx, hits_dbl_idx, hits_hf_idx = calculate_metrics_with_oe(ref, calculated, tolerance = tolerance)
        
        results["mode"].append(mode)
        results["axis"].append(axis_name)
        results["acc1"].append(metrics["acc1"])
        results["acc2"].append(metrics["acc2"]) # double/ half
        results["acc3"].append(metrics["acc3"]) # 1x 2x 3x 0.5x 0.33x
        
        results["hits_idx"].append(hits_idx.tolist())
        results["hits_dbl_idx"].append(hits_dbl_idx.tolist())
        results["hits_hf_idx"].append(hits_hf_idx.tolist())

results_df = pd.DataFrame(results)
# results_df.to_csv(f'./{root_dir}/score/{base_dir}/{marker_name}_{mode}_score.csv')

In [ ]:
results_df

In [ ]:
# Plot using Seaborn
plt.figure(figsize=(8, 5))  # Adjust figure size for better readability
sns.barplot(data=results_df, x="experiment", y="acc1", palette="muted", hue="axis")

#
plt.xlabel("Onset type")
plt.ylabel("Acc1 (%)")
plt.title(f"{base_dir} Per Axis 1S (position): Acc1 - Tempi range: 70-145")
plt.xticks(rotation=45)  # Rotate labels if necessary
plt.ylim(0, 100)  # Set consistent y-axis limits
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()
# plt.savefig(f'./{root}/{base_dir}/{base_dir}_acc1_70-145.png')
plt.show()

In [ ]:
# Plot metrics for each experiment
for exp_name in results_df["experiment"].unique():
    exp_data = results_df[results_df["experiment"] == exp_name]
    
    x = np.arange(len(exp_data))  # Axes: X, Y, Z, XYZ
    bar_width = 0.15

    plt.figure(figsize=(5, 5))
    plt.bar(x - 2 * bar_width, exp_data["acc1"], bar_width, label="Acc1 (Strict)", color="blue")
    plt.bar(x - bar_width, exp_data["acc2"], bar_width, label="Acc2 (Double/Half)", color="green")
    # plt.bar(x, exp_data["Metric3"], bar_width, label="Metric3 (Subdivisions)", color="purple")
    # plt.bar(x + bar_width, exp_data["OE1"], bar_width, label="OE1 (Strict Overestimation)", color="orange")
    # plt.bar(x + 2 * bar_width, exp_data["OE2"], bar_width, label="OE2 (Hierarchical Overestimation)", color="red")



    plt.xticks(x, exp_data["axis"])
    plt.xlabel("Axes")
    plt.ylabel("Percentage (%)")
    plt.title(f"Per Axis - Left Foot -{exp_name}")
    plt.legend()
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.ylim(0, 100)
    # plt.savefig(f'./plots/left_foot_{exp_name}.png')
    plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(ref, bins=10)